In [1]:
import tweepy
import csv
import json
import objectpath
import pandas as pd
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EmotionOptions, ConceptsOptions, CategoriesOptions,SentimentOptions

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

#Watson NLU credentials
nlu = NaturalLanguageUnderstandingV1(
    version='',
    iam_apikey='',
    url='https://gateway.watsonplatform.net/natural-language-understanding/api'
)
#------------

screen_name="realDonalTrump"
    #Twitter only allows access to a users most recent 3240 tweets with this method
    #authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
    #initialize a list to hold all the tweepy Tweets
alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
while len(new_tweets) > 0:
    print ("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
    new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
    alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    print ("...%s tweets downloaded so far" % (len(alltweets)))

getting tweets before 508924469818437631
...5 tweets downloaded so far


In [2]:
outtweets=["tweet_id","created date","tweet","tw_type","RT Count","Fav","Tone_Tweet","Tone_Tweet_score","emotion_sad",
           "emotion_joy","emotion_fear","emotion_disgust","emotion_anger","concept1","concept1_relevance",
           "concept2","concept2_relevance","category_label","category_score"]
tw_type=""
tw_body1=""
for tweet in alltweets:
    tweet_raw=tweet.text
    if 'https' in tweet_raw:
        twi=tweet_raw.split("https")
        tw_body=twi[0]
        tw_type="Link"
    if 'RT' in tw_body:
        twi=tw_body.split("RT")
        tw_body=twi[1]
        tw_type=tw_type+"RT"
    #tw_body=tweet.text.encode("utf-8")
    #Do NLU stuff on tweet
    try:
        tw = nlu.analyze(text=tw_body,
        features=Features(emotion=EmotionOptions(),
        sentiment=SentimentOptions(),
        categories=CategoriesOptions(limit=3),
        concepts=ConceptsOptions(limit=3)))
        
        Tone_Tweet=tw.result['sentiment']['document']['label']
        Tone_Tweet_score=tw.result['sentiment']['document']['score']
        emotion_sad=tw.result['emotion']['document']['emotion']['sadness']
        emotion_joy=tw.result['emotion']['document']['emotion']['joy']
        emotion_fear=tw.result['emotion']['document']['emotion']['fear']
        emotion_disgust=tw.result['emotion']['document']['emotion']['disgust']
        emotion_anger=tw.result['emotion']['document']['emotion']['anger']
        concept1=tw.result['concepts'][0]['text']
        concept1_relevance=tw.result['concepts'][0]['relevance']
        concept2=tw.result['concepts'][1]['text']
        concept2_relevance=tw.result['concepts'][1]['relevance']
        category_label=tw.result['categories'][0]['label']
        category_score=tw.result['categories'][0]['score']
    except Exception as e:
        print(e)
        '''
        Tone_Tweet='?'
        Tone_Tweet_score='?'
        emotion_sad='?'
        emotion_joy='?'
        emotion_fear='?'
        emotion_disgust='?'
        emotion_anger='?'
        concept1='?'
        concept1_relevance='?'
        concept2='?'
        concept2_relevance='?'
        category_label='?'
        category_score='?'
    '''
    #Save other params into data rows

    
    outtweets.append([tweet.id_str, tweet.created_at, tw_body, tw_type, tweet.retweet_count, tweet.favorite_count,emotion_sad,emotion_joy,emotion_fear,emotion_disgust,emotion_anger,concept1,concept1_relevance,concept2,concept2_relevance,category_label,category_score
])
print(outtweets)

NameError: name 'tw_body' is not defined

In [32]:

#write the csv
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["tweet_id","created date","tweet","tw_type","RT Count","Fav","Tone_Tweet","Tone_Tweet_score","emotion_sad","emotion_joy","emotion_fear","emotion_disgust","emotion_anger","concept1","concept1_relevance","concept2","concept2_relevance","category_label","category_score"])
        writer.writerows(outtweets)
    pass


                                                    0
0                                            tweet_id
1                                        created date
2                                               tweet
3                                             tw_type
4                                            RT Count
5                                                 Fav
6                                          Tone_Tweet
7                                    Tone_Tweet_score
8                                         emotion_sad
9                                         emotion_joy
10                                       emotion_fear
11                                    emotion_disgust
12                                      emotion_anger
13                                           concept1
14                                 concept1_relevance
15                                           concept2
16                                 concept2_relevance
17                          